# Chap03 - Word2Vec

## 3.1 추론 기반 기법과 신경망

### 3.1.1 통계 기반 기법의 문제점

통계 기반 기법에서는 주변 단어의 빈도를 기반으로 단어를 표현했다. 단어의 Co-occurrence Matrix를 만들고, 그 행렬에 SVD를 적용하여 밀집벡터(단어의 분산표현)을 구했다. 하지만 이런 방식은 대규모 말뭉치(corpus)를 다룰 때 문제가 발생한다. 

통계 기반 기법은 말뭉치 전체의 통계(동시발생 행렬과 PPMI 등)를 이용해 **단 1회의 처리**(SVD 등)만에 단어의 분산 표현을 얻는다. 하지만 신경망을 이용한 **추론 기법**에서는 미니 배치(학습 데이터의 일부)를 이용해 학습한다.

이처럼 신경망은 미니 배치(mini-batch)로 학습하기 때문에 대규모의 말뭉치에서도 학습시킬 수 있다.

### 3.1.2 추론 기반 기법 개요

추론 기법에서의 추론이란 주변 단어(맥락, context)가 주어졌을 때 "?"에 어떤 단어가 들어가는지 추측하는 것을 말한다.

![](./images/inference.png)

이러한 추론 문제를 방복해서 풀면서 단어의 출현 패턴을 학습한다. 모델은 맥락(context) 정보를 입력받아 각 단어의 출현 확률을 출력한다. '모델 관점'에서 보면 다음과 같다.

![](./images/inference02.png)



### 3.1.3 신경망에서의 단어 처리

신경망의 입력층에서의 뉴런 수를 **고정**하기 위해 단어를 일정 길이를 갖는 **고정 길이 벡터**로 변환 해준다. 이때 사용하는 대표적인 방법이 **원핫** 벡터(one-hot vector)다.

| 단어(텍스트) | 단어 ID | one-hot vector        |
| ------------ | ------- | --------------------- |
| you          | 0       | [1, 0, 0, 0, 0, 0, 0] |
| goodbye      | 2       | [0, 0, 1, 0, 0, 0, 0] |


<img src="./images/one-hot.png" height="65%" width="65%" />

위와 같이 원-핫 벡터로 나타낸 단어 벡터를 신경망의 입력으로 주입하고 완전연결계층(FC layer)를 통해 원-핫 벡터로 표현된 단어를 밀집 벡터로 변환해줄 수 있다.

![](./images/nn.png)

In [1]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]])  # 입력 (one-hot)
W = np.random.randn(7, 3)  # 가중치 (랜덤한 값), Word vector
h = np.matmul(c, W)  # 은닉층 노드
print(h)

[[ 1.08641249 -0.82780485 -0.29980432]]


위의 코드에서 `c`는 원-핫 표현이며 단어 ID에 대응하는 원소만 1이고 그 외에는 0인 벡터다. 따라서, `c`와 `W`의 행렬곱은 가중치(`W`)의 행벡터 하나를 **뽑아낸** 것과 같다.

![](./images/matmul.png)

In [4]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-0.38687194  0.82963359  0.87623966]]


## 3.2 단순한 Word2Vec